# Select reservoirs
***

**_Autor:_** Chus Casado Rodríguez<br>
**_Fecha:_** 01-08-2024<br>

**Introduction:**<br>
This code compares the available attributes for reservoirs in Spain from different sources: the Spanish Ministry, ICOLD, GRanD.

**To be checked:**<br>
* [ ] The field `cedex_dam.Qspill_EFL` is mostly empty.
* [x] There's a erroneous reservoir in EFAS. "El Alisillo" (ResID=3247) does not refer to any real reservoir, but it's very closely located to the "Montoro III" reservoir. The attributes don't much, though, so it's not a matter of just renaming the reservoir. <font color='steelblue'>I have created a new reservoir to represent "Montoro III"</font>.
* [ ] How to proceed when there are several reservoirs in a continuous chain. For instance, Torrejón-Tajo and Arrocampo, or Tanes and Rioseco. Should we simulate each of them, or simply simulate them as a single reservoir with the aggregated volume?
* [x] Attributes from GRanD
* [x] Compare attributes among data sources: reservoir volume, etc.
* [x] Should I include in the tables of attributes only the reservoirs with time series, or all the reservoirs? <font color='steelblue'>I have included all the reservoirs, so they can be later on used for regionalization, i.e., to estimate their reservoir parameters based on the attributes even though there aren't records.
* [ ] Filter reservoirs by the length of their observed time series (for instance, longer than 8 years)

In [1]:
import os
os.environ['USE_PYGEOS'] = '0'
from pathlib import Path
# import sys
# sys.path.append('../../src/')
import geopandas as gpd
from shapely.geometry import Point
import re

import numpy as np
import pandas as pd
import xarray as xr
from datetime import datetime, timedelta
from tqdm.auto import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import seaborn as sns
sns.set_style('ticks')
import yaml

In [2]:
from lisfloodreservoirs.utils import DatasetConfig
from lisfloodreservoirs.utils.SNCZI import reservoir_attributes, dam_attributes
from lisfloodreservoirs.utils.names import remove_accents, correct_names
from lisfloodreservoirs.utils.utils import filter_reservoirs, remove_duplicates

In [3]:
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

## Configuration

In [5]:
cfg = DatasetConfig('config_dataset.yml')

# path where plots will be saved
PATH_PLOTS = cfg.PATH_ATTRS / 'plots'
PATH_PLOTS.mkdir(parents=True, exist_ok=True)

# path of the Spanish Inventory of Dams and Reservoirs
PATH_DATASETS = Path('Z:/nahaUsers/casadje/datasets')
PATH_SNCZI = PATH_DATASETS / 'SNCZI' / 'reports' 

## CEDEX

### Reservoir attributes

There are several attributes of interest in this table:

* Coordinates.
* Reservoir volume: both total and live volume.
* Reservoir area.
* Design reservoir levels.


In [ ]:
# load reservoir attributes
cedex_res = pd.read_csv(PATH_SNCZI / 'reservoirs' / 'attributes_reservoirs.csv', index_col='res_ID')
cedex_res.inf_ID = cedex_res.inf_ID.astype('Int64')

# rename columns
rename_cols = {'Código del embalse': 'res_ID',
               'Nombre del embalse': 'RES_NAME',
               'Titular del embalse': 'owner',
               'Coord. X ETRS89': 'LON', # longitude
               'Coord. Y ETRS89': 'LAT', # latitud
               'Coord. X UTM ETRS89 Huso 30': 'X',
               'Coord. Y UTM ETRS89 Huso 30': 'Y',
               'Coord. X Manual': 'X_manual',
               'Coord. Y Manual': 'Y_manual',
               'Volumen útil (m3)': 'V_live',
               'Volumen total (m3)': 'CAP_MCM',
               'Superficie del embalse (has)': 'AREA_SKM',
               'Máximo nivel de avenida (m)': 'Z_EFL', # Extreme Flood Level
               'Máximo nivel normal del embalse (m)': 'Z_MNL', # Maximum Normal Level
               'Municipio': 'town', 
               'Id. Hoja 1:50.000': 'sheet_MTN50',
               'Cauce': 'river', 
               'Código de infraestructura': 'inf_ID', 
               'Nombre de la presa': 'DAM_NAME'}
cedex_res.rename(columns=rename_cols, inplace=True)

# correct names
for col in ['RES_NAME', 'DAM_NAME']:
    cedex_res[col] = cedex_res[col].replace(np.nan, '')
    cedex_res[col] = [remove_accents(name) for name in cedex_res[col]]

# convert reservoir area to km2
cedex_res.AREA_SKM /= 100

# remove duplicated entries keeping the largest reservoir
remove_duplicates(cedex_res, 'RES_NAME', 'CAP_MCM')
        
print('Imported from SNCZI {0} reservoirs and {1} attributes'.format(*cedex_res.shape))

### Dam attributes

This dataset contains multiple attributes of interes:

* Date of construction.
* Coordinates.
* Reservoir use.
* Catchment area.
* Reservoir level, surface and volume at the maximum normal level (MNL).
* Elevation of river, foundation, crest.
* Number of outlets (spillways and sluices) and their discharge capacity.

In [ ]:
# import dam attributes
cedex_dam = pd.read_csv(PATH_SNCZI / 'dams' / 'attributes_dams.csv', index_col='inf_ID')

# rename columns
rename_cols = {'Nombre de la presa': 'DAM_NAME',
               'Otro Nombre': 'other_name',
               'Fase vida presa': 'phase',
               'Titular de la presa': 'owner',
               'Proyectista': 'designer',
               'Categoría en función del riesgo potencial': 'category',
               'Aprobación de las normas de explotación': 'date_exploitation_plan',
               'Aprobación del plan de emergencia': 'date_emergency_plan',
               'Fecha de finalización de las obras': 'date_construction',
               'Río en el que se encuentra la presa': 'river', 
               'Municipio': 'town',
               'Cuenca hidrográfica': 'basin', 
               'Provincia': 'province', 
               'X-UTM30ETRS89': 'X', 
               'Y-UTM30ETRS89': 'Y',
               'Usuarios': 'users', 
               'Tipos': 'use', 
               'Superficie de la cuenca hidrográfica (km2)': 'CATCH_SKM',
               'Aportación media anual (hm3)': 'INFLOW_MCM', 
               'Precipitación media anual (mm)': 'PREC_MM',
               'Caudal punta avenida de proyecto (m3/s)': 'Q_design',
               'Superficie del embalse a NMN (ha)': 'AREA_MNL', # Nivel Máximo Normal => Maximum Normal Level
               'Capacidad a NMN (hm3)': 'VOL_MNL',
               'Cota del NMN (m)': 'Z_MNL', 
               'Tipo de presa': 'type', 
               'Cota coronación (m)': 'ELEV_MASL',
               'Altura desde cimientos (m)': 'DAM_HGT_M', 
               'Longitud de coronación (m)': 'len_crest',
               'Cota cimentación (m)': 'Z_foundation', 
               'Cota del cauce en la presa (m)': 'Z_river',
               'Volumen del cuerpo presa (1000 m3)': 'dam_vol',
               'Número total de aliviaderos en la presa': 'no_spill', 
               'Regulación': 'regulation',
               'Capacidad a NAE (m3/s)': 'Qspill_EFL', # Nivel de Avenida Extrema => Extreme Flood Level
               'Número total de desagües en la presa': 'no_sluice',
               'Capacidad (m3/s)': 'Qsluice'}
cedex_dam.rename(columns=rename_cols, inplace=True)

# correct names
for col in ['DAM_NAME']:
    cedex_dam[col] = cedex_dam[col].replace(np.nan, '')
    cedex_dam[col] = [remove_accents(name) for name in cedex_dam[col]]
    
# convert to date
for col in cedex_dam.columns[cedex_dam.columns.str.startswith('date')]:
    cedex_dam[col] = pd.to_datetime(cedex_dam[col], format='%d-%m-%Y', errors='coerce')

# convert reservoir area to km2
cedex_dam.AREA_MNL /= 100

# remove duplicate dams and keep the highest
remove_duplicates(cedex_dam, 'DAM_NAME', 'DAM_HGT_M')

print('{0} dams with {1} attributes imported from SNCZI'.format(*cedex_dam.shape))

# assign reservoir ID to each dam
res_ID = []
for inf_ID in cedex_dam.index:
    try:
        res_id = cedex_res.loc[cedex_res.inf_ID == inf_ID].index[0]
    except:
        res_id = None
    res_ID.append(res_id)
cedex_dam['res_ID'] = res_ID
cedex_dam.res_ID = cedex_dam.res_ID.astype('Int64')

# convert into GeoDataFrame
geometry = [Point(xy) for xy in zip(cedex_dam.X, cedex_dam.Y)]
cedex_dam = gpd.GeoDataFrame(cedex_dam, geometry=geometry, crs='epsg:25830')
cedex_dam.to_crs(epsg=4326, inplace=True)
cedex_dam['LON'] = cedex_dam.geometry.x
cedex_dam['LAT'] = cedex_dam.geometry.y

print('{0} out of {1} dams have an associated reservoir'.format((~cedex_dam.res_ID.isnull()).sum(), cedex_dam.shape[0]))

#### Reservoir use

**Create individual colums for each use**

I convert the `use` column into a one hot encoder.

In [ ]:
# analyse reservoir use:
# 1. I will suppose that the the first use in the dataset is the `main_use`
# 2. I create a `single_use` field to identify single-purpose and multipurpose reservoirs
# 3. I create a one hot encoder class with all the possible reservoir uses

cedex_dam.use = cedex_dam.use.replace(np.nan, '')
rename_uses_cedex = {'': 'other',
                     'Abastecimiento': 'supply',
                     'Acuicultura': 'fish',
                     'Adecuación ambiental': 'environment',
                     'Defensa frente avenidas': 'flood',
                     'Ganadero': 'livestock',
                     'Hidroeléctrico': 'hydropower',
                     'Industrial': 'industry',
                     'Minería': 'mining',
                     'Recreo': 'recreation',
                     'Refrigeración': 'cooling',
                     'Regulación': 'regulation',
                     'Riego': 'irrigation',
                     'Trasvase': 'diversion'}

cedex_dam['main_use'] = ''
cedex_dam['single_use'] = True
cedex_uses = list(rename_uses_cedex.values())
# cedex_uses.remove('')
cedex_uses.sort()
cedex_dam[cedex_uses] = False
for ID in cedex_dam.index:
    uses_i = cedex_dam.loc[ID, 'use'].split('. ')
    # if '' in uses_i:
    #     uses_i.remove('')
    if len(uses_i) > 0:
        for i, use in enumerate(uses_i):
            use = rename_uses_cedex[use]
            if i == 0:
                cedex_dam.loc[ID, 'main_use'] = use
            cedex_dam.loc[ID, use] = True

# identify multi-purpose reservoirs
mask_multiuse = cedex_dam[cedex_uses].sum(axis=1) > 1
cedex_dam.loc[mask_multiuse, 'single_use'] = False

# remove original 'use' field
# cedex_dam.drop('use', axis=1, inplace=True)

**Classify hydropower reservoirs**

I will create a field called `HP_type` (hydropower type) to specify the importance of the hydropower use among all the uses of that reservoir. Codes:

* 0: single purpose hydropower.
* 1: multi-purpose in which hydropower is the main use.
* 2: multi-purpose in which hydropower is the second use.
* 3: multi-purpose in which hydropower is the third use.
* N: multi-purpose in which hydropower is the Nth use.

```Python
cedex_dam['HP_type'] = np.nan

cedex_dam.loc[cedex_dam.single_use & cedex_dam.hydropower, 'HP_type'] = 0

for ID in cedex_dam.loc[cedex_dam.hydropower].index:
    if cedex_dam.loc[ID, 'single_use']:
        continue
    uses = [rename_uses_cedex[use] for use in cedex_dam.loc[ID, 'use'].split('. ')]
    if 'hydropower' in uses:
        cedex_dam.loc[ID, 'HP_type'] = uses.index('hydropower') + 1

cedex_dam.HP_type.value_counts()
 ```

**Total annual volume outflow (hm3)**

```Python
outflows = pd.DataFrame(dtype=float)
for ID in tqdm(cedex_dam.loc[cedex_dam.hydropower].index):
    
    # load time series
    df = pd.read_csv(path_cedex / 'timeseries' / f'{ID}.csv', index_col='date')
    df.index = pd.to_datetime(df.index)
    if 'outflow' not in df.columns:
        continue
    
    # compute annual total flow
    outflow_y = df.loc[datetime(2010, 1, 1): datetime(2019, 12, 31), 'outflow'].resample('y').mean() * 365 * 24 * 3600 / 1e6
    outflow_y.index = [f'outflow_{idx.year}' for idx in outflow_y.index]
    outflow_y = outflow_y.to_frame(ID)
    
    # concatenate
    outflows = pd.concat((outflows, outflow_y), axis=1)
```

### Combine reservoir and dam attributes
The connection is direct using the common `res_ID` field.

In [ ]:
# rename index
cedex_res.index.name = 'SNCZI'

# set res_ID as index
cedex_dam.reset_index(inplace=True)
cedex_dam.set_index('res_ID', drop=True, inplace=True)
cedex_dam.index.name = 'SNCZI'

In [ ]:
# columns to be used from the inventory of dams in CEDEX
cols_cedex_dam = ['DAM_NAME', 'LON', 'LAT', 'date_construction',
                  # catchment hydrology
                  'CATCH_SKM', 'INFLOW_MCM', 'PREC_MM', 'Q_design', 
                  # reservoir geometry
                  'AREA_MNL', 'VOL_MNL', 'Z_MNL', 'ELEV_MASL', 'DAM_HGT_M', #'dam_vol', 
                  # reservoir regulation
                  'no_spill', 'Qspill_EFL', 'regulation', 'no_sluice', 'Qsluice',
                  # reservoir use
                  'main_use', 'single_use', 'fish', 'diversion', 'environment', 'flood', 'hydropower', 'industry', 'irrigation', 'livestock',
                  'mining', 'cooling', 'other', 'recreation', 'supply']

# columns to be used from the inventory of reservoirs in CEDEX
cols_cedex_res = ['RES_NAME', 'V_live', 'CAP_MCM', 'AREA_SKM', 'Z_EFL', 'Z_MNL']

# combine attributes from CEDEX
cedex = pd.concat((cedex_dam.loc[~cedex_dam.index.isnull(), cols_cedex_dam],
                   cedex_res[cols_cedex_res]),
                  axis=1,
                  join='outer')
cedex.index = cedex.index.astype(int)
cedex.sort_index(inplace=True)
cedex.DAM_NAME = cedex.DAM_NAME.replace(np.nan, '')

# convert into GeoDataFrame
geometry = [Point(xy) for xy in zip(cedex.LON, cedex.LAT)]
cedex = gpd.GeoDataFrame(cedex, geometry=geometry, crs='epsg:4326')

# correct some names
cedex.RES_NAME = cedex.RES_NAME.replace({'SANT PONC (SAN PONS)' : 'SAN PONS',
                                         'CATLLAR': 'GAIA'})
cedex.DAM_NAME = cedex.DAM_NAME.replace({'SANT PONC': 'SAN PONS',
                                         'RIO JILOCA (REGULACION) (LECHAGO)': 'LECHAGO',
                                         'CATLLAR': 'GAIA'})

# remove zeros
cedex[['CAP_MCM', 'CATCH_SKM']] = cedex[['CAP_MCM', 'CATCH_SKM']].replace(0, np.nan)

# filter by catchment area and reservoir volume
mask_cedex = filter_reservoirs(cedex.CATCH_SKM, cedex.CAP_MCM, MIN_CATCH, MIN_VOL)

fig, ax = plt.subplots(subplot_kw=dict(projection=ccrs.PlateCarree()))
ax.add_feature(cfeature.NaturalEarthFeature('physical', 'land', '10m', edgecolor='face', facecolor='lightgray'),
               alpha=.5,
               zorder=0)
cedex.loc[~mask_cedex].plot(markersize=cedex.CAP_MCM**.5, c='indianred', alpha=.5, ax=ax, label='discarded')
cedex.loc[mask_cedex].plot(markersize=cedex.CAP_MCM**.5, alpha=.5, ax=ax, label='selected')
ax.set_extent([-10, 4.5, 35.5, 44])
fig.legend(frameon=False, bbox_to_anchor=[1.0, .4, .1, .2])
ax.axis('off');

**Main use**

In [ ]:
mainuse_singlepurpose = cedex.loc[mask_cedex & cedex.single_use, 'main_use'].value_counts()
mainuse_multipurpose = cedex.loc[mask_cedex, 'main_use'].value_counts()
mainuse_summary_cedex = pd.concat((mainuse_singlepurpose, mainuse_multipurpose), axis=1)
mainuse_summary_cedex.columns = ['singlepurpose', 'multipurpose']
mainuse_summary_cedex.sort_values('multipurpose', ascending=False, inplace=True)
del mainuse_singlepurpose, mainuse_multipurpose

**All uses**

In [ ]:
alluses_singlepurpose = cedex.loc[mask_cedex & cedex.single_use, cedex_uses].sum()
alluses_multipurpose = cedex.loc[mask_cedex, cedex_uses].sum()
alluses_summary_cedex = pd.concat((alluses_singlepurpose, alluses_multipurpose), axis=1)
alluses_summary_cedex.columns = ['singlepurpose', 'multipurpose']
alluses_summary_cedex.sort_values('multipurpose', ascending=False, inplace=True)
alluses_summary_cedex.dropna(axis=0, how='all', inplace=True)
alluses_summary_cedex = alluses_summary_cedex[~(alluses_summary_cedex == 0).all(axis=1)]
del alluses_singlepurpose, alluses_multipurpose

## EFAS

In [ ]:
# load EFAS reservoirs
efas = gpd.read_file(PATH_EFAS / 'reservoirs' / 'reservoirs_EFAS5.shp')
efas.ResID = efas.ResID.astype(pd.Int64Dtype())
efas.set_index('ResID', inplace=True)

# filter reservoir in Spain
efas = efas.loc[efas.COUNTRY == 'Spain']

# correct dam names
efas.DAM_NAME.replace(np.nan, '', inplace=True)
names = []
for name in efas.DAM_NAME.str.upper():
    split = name.split('_')
    if len(split) == 2:
        names.append(split[0])
    elif len(split) > 2:
        names.append(' '.join(split[:-1]))
    else:
        names.append(name)
efas.DAM_NAME = names

# correct coordinates of Cenza reservoir
# efas.loc[3132, ['LON_ORG', 'LAT_ORG']] = [-7.1442, 42.1141]

# remove old reservoir "El Alisillo" (ResID = 3247)
efas.drop([3247], axis=0, inplace=True)

# convert coordinates into the original
efas = pd.DataFrame(efas)
efas[['LON_ORG', 'LAT_ORG']] = efas[['LON_ORG', 'LAT_ORG']].astype(float)
geometry = [Point(xy) for xy in zip(efas.LON_ORG, efas.LAT_ORG)]
efas = gpd.GeoDataFrame(efas, geometry=geometry, crs='epsg:4326')
efas.rename(columns={'LON_ORG': 'LON', 'LAT_ORG': 'LAT'}, inplace=True)

# add EFAS reservoir parameters
EFAS_vars = {'rclim': 'Vc',
             'rflim': 'Vf',
             'rminq': 'Qmin',
             'rndq': 'Qnd',
             'rnlim': 'Vn',
             'rnormq': 'Qn',
             'rtstor': 'CAP_MCM'}

for file in (PATH_EFAS / 'reservoirs').glob('r*.txt'):
    var = EFAS_vars[file.stem]
    serie = pd.read_csv(file, header=None, index_col=0, sep=' ').squeeze()
    efas[var] = serie.loc[efas.index]
# convert volumes to hm3
efas['CAP_MCM'] /= 1e6

I load the static map `res.nc` to extract from it the coordinates of the reservoirs in EFAS.

In [ ]:
# load static map of reservoirs
efas_raster = xr.open_mfdataset(f'{PATH_EFAS}/iberia/res*.nc')['res'].compute()

# extract an array of reservoir ID
# ids = np.unique(glofas_raster.where(~glofas_raster.isnull(), drop=True))
# ids = ids[~np.isnan(ids) & (ids != -9999)]
# ids = ids.astype(int)
ids = efas.index.sort_values().tolist()

# xr.DataArrays of reservoir longitudes and latitudes
lon = xr.DataArray(np.nan, dims=['ResID'], coords={'ResID': ids})
lat = xr.DataArray(np.nan, dims=['ResID'], coords={'ResID': ids})
for id in tqdm(ids):
    try:
        cell = efas_raster.where(efas_raster == id, drop=True)
        lon.loc[dict(ResID=id)] = cell.lon.data[0]
        lat.loc[dict(ResID=id)] = cell.lat.data[0]
    except:
        lon = lon.where(lon.ResID != id, drop=True)
        lat = lat.where(lat.ResID != id, drop=True)
coords = xr.Dataset({'lon': lon, 'lat': lat})

In [ ]:
# extract catchment area from the "upArea" static map
upArea = xr.open_mfdataset(f'{PATH_EFAS}/iberia/upArea*.nc')['Band1'].compute()
upArea.name = 'upstream_area'
efas['CATCH_SKM'] = np.nan
for ID in tqdm(efas.index):
    try:
        lon, lat = coords.sel(ResID=ID)['lon'].data, coords.sel(ResID=ID)['lat'].data
        efas.loc[ID, 'CATCH_SKM'] = upArea.sel(lon=lon, lat=lat, method='nearest').drop(['lon', 'lat']).data
    except:
        print(f'The catchment area could not be extracted for ResID {ID}')
efas.CATCH_SKM /= 1e6 # convert into km2

In [ ]:
#  select reservoirs based on catchment area and reservoir volume
mask_efas = filter_reservoirs(efas.CATCH_SKM, efas.CAP_MCM, MIN_CATCH, MIN_VOL)

fig, ax = plt.subplots(subplot_kw=dict(projection=ccrs.PlateCarree()))
ax.add_feature(cfeature.NaturalEarthFeature('physical', 'land', '10m', edgecolor='face', facecolor='lightgray'),
               alpha=.5,
               zorder=0)
efas.loc[~mask_efas].plot(markersize=efas.CAP_MCM**.5, c='indianred', alpha=.5, ax=ax, label='discarded')
efas.loc[mask_efas].plot(markersize=efas.CAP_MCM**.5, alpha=.5, ax=ax, label='selected')
# ax.set_extent([-10, 4.5, 35.5, 44])
fig.legend(frameon=False, bbox_to_anchor=[1.0, .4, .1, .2])
ax.axis('off');

# apply the selection
efas = efas.loc[mask_efas]

There's a reservoir missing in the EFAS raster (`ResID=3132`, Cenza). Apart from that, the reservoir Montoro III that I have created to replace "El Alisillo" is also missing.

### Connect EFAS and CEDEX

In [ ]:
# create fields for the ID and name in HDCC
efas[['SNCZI', 'SNCZI_Name']] = np.nan

# find matches based on the name
mask_name = cedex.RES_NAME.isin(efas.DAM_NAME)
for ID, name in cedex.loc[mask_name, 'RES_NAME'].items():
    efas.loc[efas.DAM_NAME == name, ['SNCZI', 'SNCZI_Name']] = ID, name

# check missing values
missing = efas.SNCZI.isnull().sum()
if missing > 0:
    print(f'{missing} reservoir are missing the SNCZI connection')
    efas.SNCZI = efas.SNCZI.astype('Int64')
else:
    efas.SNCZI = efas.SNCZI.astype(int)
    
# reindex
efas.reset_index(inplace=True)
efas.set_index('SNCZI', drop=True, inplace=True)
efas.sort_index(inplace=True)

## Hydropower Data Base (HPDB)

```Python
hpdb = pd.read_excel(PATH_DATASETS / 'reservoirs' / 'HPDB' / 'HPDB_Spain.xlsx', index_col='HPDB_id')
hpdb.Name = hpdb.Name.str.upper()
# # correct coordinates of "La Muela"
# hpdb.loc['H4', ['Longitude', 'Latitude']] = [-0.92821, 39.23821]

# rename columns
rename_cols = {#'Name',
               # 'country',
               'Longitude': 'lon',
               'Latitude': 'lat',
               'HPDB_installed_capacity_MW': 'instal_cap',
               'HPDB_pumping  capacity if it is a HPHS_MW': 'pump_cap',
               'HPDB_type': 'type',
               'HPDB_head_m': 'head_HPDB',
               'head': 'head_src2',
               'dam height (match from ICOLD o Vattenfal for Sweden)': 'DAM_HGT_M',
               'head (final)': 'head',
               'HPDB_volume_Mm3': 'vol_HPDB',
               'HPDB_volume_from ICOLD_Mm3': 'vol_ICOLD',
               'HPDB_volume of reservoir_final_Mm3': 'volume',
               'final energy generation GWh': 'energy',
               'Annual volume discharged (a bit underestimated, because the used head is the maximum one) Mm3': 'vol_turb'
              }
hpdb.rename(columns=rename_cols, inplace=True)

# filter dam hydropower plants
hpdb = hpdb.loc[hpdb.type == 'HDAM']

# convert into GeoDataFrame
geometry = [Point(xy) for xy in zip(hpdb.lon, hpdb.lat)]
hpdb = gpd.GeoDataFrame(hpdb, geometry=geometry, crs='epsg:4326')

fig, ax = plt.subplots(subplot_kw=dict(projection=ccrs.PlateCarree()))
ax.add_feature(cfeature.NaturalEarthFeature('physical', 'land', '10m', edgecolor='face', facecolor='lightgray'),
               alpha=.5,
               zorder=0)
hpdb.plot(markersize=5, ax=ax)
ax.set_title(f'Hydropower Batabase\n{hpdb.shape[0]} reservoirs')
ax.set_extent([-10, 4.5, 35.5, 44])
ax.axis('off');
```

**Connect HPDB with HDCC**

```Python
# nearest neighbour in ICOLD for each point in HDCC
idx_hpdb, idx_hdcc = hdcc.sindex.nearest(hpdb.geometry, max_distance=.1)

# add ICOLD name and ID to the HDCC table
hpdb.drop(['HDCC_ID', 'HDCC_Name'], axis=1, inplace=True, errors='ignore')
idx_hpdb = hpdb.index[idx_hpdb]
hpdb.loc[idx_hpdb, 'HDCC_ID'] = hdcc.iloc[idx_hdcc].index.values
hpdb.loc[idx_hpdb, 'HDCC_Name'] = hdcc.iloc[idx_hdcc]['Name'].values

(hpdb.Name == hpdb.HDCC_Name).sum()

mask_name = hpdb.Name != hpdb.HDCC_Name
hpdb.loc[mask_name, ['Name', 'HDCC_Name']].sort_values('Name').head(30)

for col in hpdb.columns:
    if len(col) > 10:
        print(col)

hpdb.to_file(PATH_DATASETS / 'reservoirs' / 'HPDB' / 'HPDB_Spain.shp')

for ID in cedex_dam.index:
    name = cedex_dam.loc[ID, 'DAM_NAME']
    if 'MUELA' in name:
        print(ID, name)
```

## ICOLD

In [ ]:
# import shapefile
path_icold = PATH_DATASETS / 'reservoirs' / 'ICOLD' / '2023'
icold = gpd.read_file(path_icold / 'ICOLD_ES.shp')
icold.reset_index(drop=True, inplace=True)

# remove duplicated entries keeping the heighest dam associated to each reservoir
for name, count in icold.Res_Name.value_counts().items():
    if count > 1:
        remove_idx = icold.loc[icold.Res_Name == name].sort_values('Height', ascending=False).index[1:]
        icold.drop(remove_idx, axis=0, inplace=True)
    else:
        break
        
# remove entries without geometry
icold.drop([idx for idx, geo in icold.geometry.items() if geo is None], axis=0, inplace=True)
        
# convert reservoir volume MCM (hm3)
icold.Res_Vol /= 1000

# convert reservoir area to km2
icold.Res_Area /= 1000

# rename columns
icold.rename(columns={'Catch_Area': 'CATCH_SKM',
                      'Res_Vol': 'CAP_MCM',
                      'Res_Area': 'AREA_SKM',
                      'Res_Name': 'RES_NAME',
                      'Res_Leng': 'RES_LEN_KM',
                      'Dam_Name': 'DAM_NAME',
                      'Latitude': 'LAT',
                      'Longitude': 'LON',
                      'Height': 'DAM_HGT_M',
                      'Altitude': 'ELEV_MASL',
                      'Crest_Leng': 'DAM_LEN_M'},
             inplace=True)

# correct names
icold = correct_names(icold, col_pattern='name', split_pattern=', ')

# select reservoirs based on catchment area and reservoir volume
mask_icold = filter_reservoirs(icold.CATCH_SKM, icold.CAP_MCM, MIN_CATCH, MIN_VOL)

fig, ax = plt.subplots(subplot_kw=dict(projection=ccrs.PlateCarree()))
ax.add_feature(cfeature.NaturalEarthFeature('physical', 'land', '10m', edgecolor='face', facecolor='lightgray'),
               alpha=.5,
               zorder=0)
icold.loc[~mask_icold].plot(markersize=icold.CAP_MCM**.5, c='indianred', alpha=.5, ax=ax, label='discarded')
icold.loc[mask_icold].plot(markersize=icold.CAP_MCM**.5, alpha=.5, ax=ax, label='selected')
# ax.set_extent([-10, 4.5, 35.5, 44])
ax.legend(frameon=False)
ax.axis('off');

# apply the selection
icold = icold.loc[mask_icold]

### Connect CEDEX and ICOLD

In [ ]:
# create fields for the ID and name in HDCC
icold[['SNCZI', 'SNCZI_Name']] = np.nan

# find matches based on the reservoir name
mask_name = cedex.RES_NAME.isin(icold.RES_NAME)
for ID, name in cedex.loc[mask_name, 'RES_NAME'].items():
    icold.loc[icold.RES_NAME == name, ['SNCZI', 'SNCZI_Name']] = ID, name

# find matches based on the dam name
mask_snczi = icold.SNCZI.isnull()
mask_name = cedex.RES_NAME.isin(icold.loc[mask_snczi, 'DAM_NAME'])
for ID, name in cedex.loc[mask_name, 'RES_NAME'].items():
    icold.loc[icold.DAM_NAME == name, ['SNCZI', 'SNCZI_Name']] = ID, name
    
# find matches based on the dam name
mask_snczi = icold.SNCZI.isnull()
mask_name = cedex.DAM_NAME.isin(icold.loc[mask_snczi, 'DAM_NAME'])
for ID, name in cedex.loc[mask_name, 'DAM_NAME'].items():
    icold.loc[icold.DAM_NAME == name, ['SNCZI', 'SNCZI_Name']] = ID, name
    
# check missing values
missing = icold.SNCZI.isnull().sum()
if missing > 0:
    print(f'{missing} reservoir are missing the SNCZI connection')
    icold.SNCZI = icold.SNCZI.astype('Int64')
else:
    icold.SNCZI = icold.SNCZI.astype(int)
    
# reindex
#icold.reset_index(inplace=True)
#icold.set_index('SNCZI', drop=True, inplace=True)
#icold.sort_index(inplace=True)

```Python
icold[['SNCZI', 'CEDEX_name']] = np.nan
for snczi, name in tqdm(hdcc.name.items()):
    
    one = process.extractOne(name, icold.Res_Name, score_cutoff=90)

    if one is None:
        continue

    # compute distance
    point1 = hdcc.loc[hdcc.name == name].geometry.iloc[0]
    point2 = icold.loc[icold.Res_Name == one[0]].geometry.iloc[0]
    dist = geodesic((point1.y, point1.x), (point2.y, point2.x)).meters
    if dist < 1e4:
        icold.loc[icold.Res_Name == one[0], ['SNCZI', 'CEDEX_name']] = snczi, name
        
print('{0} matches found'.format((~icold.SNCZI.isnull()).sum()))
```

### Reservoir use

In [ ]:
# # count uses
# uses = []
# for ID in icold.index:
#     try:
#         uses = uses + list(icold.loc[ID, 'Purposes'])
#     except:
#         continue

# np.unique(uses, return_counts=True)

In [ ]:
rename_uses_icold = {'C': 'flood',
               'I': 'irrigation',
               'H': 'hydropower',
               'F': 'fish',
               'N': 'navigation',
               'R': 'recreation',
               'S': 'supply',
               'X': 'other',
               'T': 'tailings'}

icold['main_use'] = ''
icold['single_use'] = True

icold_uses = list(rename_uses_icold.values())
icold_uses.sort()
icold[icold_uses] = False

for ID in icold.index:
    try:
        for i, key in enumerate(icold.loc[ID, 'Purposes']):
            use = rename_uses_icold[key]
            if i == 0:
                icold.loc[ID, 'main_use'] = use
            icold.loc[ID, use] = True
    except Exception as e:
        print(f'Uses could not be renamed in reservoir {ID}: {e}')
        continue

# identify multi-purpose reservoirs
mask_multipurpose = icold[icold_uses].sum(axis=1) > 1
icold.loc[mask_multipurpose, 'single_use'] = False

# # remove original 'Purposes' field
# icold.drop('Purposes', axis=1, inplace=True)

**Main use**

In [ ]:
mask_singleuse = icold.single_use
mask_hdcc = ~icold.SNCZI.isnull()
mainuse_singlepurpose = icold.loc[mask_hdcc & mask_singleuse, 'main_use'].value_counts()
mainuse_multipurpose = icold.loc[mask_hdcc, 'main_use'].value_counts()
mainuse_summary_icold = pd.concat((mainuse_singlepurpose, mainuse_multipurpose), axis=1)
mainuse_summary_icold.columns = ['singlepurpose', 'multipurpose']
mainuse_summary_icold.sort_values('multipurpose', ascending=False, inplace=True)
del mainuse_multipurpose, mainuse_singlepurpose

**All uses**

In [ ]:
alluses_singlepurpose = icold.loc[mask_hdcc & mask_singleuse, icold_uses].sum()
alluses_multipurpose = icold.loc[mask_hdcc, icold_uses].sum()
alluses_summary_icold = pd.concat((alluses_singlepurpose, alluses_multipurpose), axis=1)
alluses_summary_icold.columns = ['singlepurpose', 'multipurpose']
alluses_summary_icold.sort_values('multipurpose', ascending=False, inplace=True)
alluses_summary_icold.dropna(axis=0, how='all', inplace=True)
alluses_summary_icold = alluses_summary_icold[~(alluses_summary_icold == 0).all(axis=1)]
del alluses_singlepurpose, alluses_multipurpose

## GRanD 

In [ ]:
# load shapefile of dams
path_grand = PATH_DATASETS / 'reservoirs' / 'GRanD' / 'v1_3'
grand = gpd.read_file(path_grand / 'GRanD_dams_v1_3.shp')
grand.set_index('GRAND_ID', drop=False, inplace=True)

# filter dams in Spain
grand = grand.loc[grand.COUNTRY == 'Spain']

# correct names
grand.DAM_NAME = grand.DAM_NAME.str.upper()
grand.RES_NAME = grand.RES_NAME.str.upper()

# rename columns
grand['LON'] = grand.geometry.x
grand['LAT'] = grand.geometry.y

print('No. reservoirs:\t{0}\nNo. attributes:\t{1}\n'.format(*grand.shape))

# select reservoirs based on catchment area and reservoir volume
mask_grand = filter_reservoirs(grand.CATCH_SKM, grand.CAP_MCM, MIN_CATCH, MIN_VOL)

fig, ax = plt.subplots(subplot_kw=dict(projection=ccrs.PlateCarree()))
ax.add_feature(cfeature.NaturalEarthFeature('physical', 'land', '10m', edgecolor='face', facecolor='lightgray'),
               alpha=.5,
               zorder=0)
grand.loc[~mask_grand].plot(markersize=grand.CAP_MCM**.5, c='indianred', alpha=.5, ax=ax, label='discarded')
grand.loc[mask_grand].plot(markersize=grand.CAP_MCM**.5, alpha=.5, ax=ax, label='selected')
# ax.set_extent([-10, 4.5, 35.5, 44])
fig.legend(frameon=False, bbox_to_anchor=[1, .4, .1, .2])
ax.axis('off');

# apply the selection
grand = grand.loc[mask_grand]

### Connect CEDEX and GRanD

In [ ]:
# create fields for the ID and name in HDCC
grand[['SNCZI', 'SNCZI_Name']] = np.nan

# find matches based on the reservoir name
mask_name = cedex.RES_NAME.isin(grand.RES_NAME)
for ID, name in cedex.loc[mask_name, 'RES_NAME'].items():
    grand.loc[grand.RES_NAME == name, ['SNCZI', 'SNCZI_Name']] = ID, name

# find matches based on the dam name
mask_snczi = grand.SNCZI.isnull()
mask_name = cedex.DAM_NAME.isin(grand.loc[mask_snczi, 'DAM_NAME'])
for ID, name in cedex.loc[mask_name, 'DAM_NAME'].items():
    grand.loc[grand.DAM_NAME == name, ['SNCZI', 'SNCZI_Name']] = ID, name  

# find matches based on the dam name
mask_snczi = grand.SNCZI.isnull()
mask_name = cedex.RES_NAME.isin(grand.loc[mask_snczi, 'DAM_NAME'])
for ID, name in cedex.loc[mask_name, 'RES_NAME'].items():
    grand.loc[grand.DAM_NAME == name, ['SNCZI', 'SNCZI_Name']] = ID, name
    
# check missing values
missing = grand.SNCZI.isnull().sum()
if missing > 0:
    print(f'{missing} reservoir are missing the SNCZI connection')
    grand.SNCZI = grand.SNCZI.astype('Int64')
else:
    grand.SNCZI = grand.SNCZI.astype(int)
    
# reindex
grand.set_index('SNCZI', drop=True, inplace=True)

```Python
grand[['SNCZI', 'SNCZI_name']] = np.nan
for snczi, name in tqdm(cedex.RES_NAME.items()):
    
    one = process.extractOne(name, grand.DAM_NAME, score_cutoff=80)

    if one is None:
        continue

    # compute distance
    point1 = cedex.loc[cedex.RES_NAME == name].geometry.iloc[0]
    point2 = grand.loc[grand.DAM_NAME == one[0]].geometry.iloc[0]
    dist = geodesic((point1.y, point1.x), (point2.y, point2.x)).meters
    if dist < 1e4:
        grand.loc[grand.DAM_NAME == one[0], ['SNCZI', 'SNCZI_name']] = snczi, name
        
print('{0} matches found'.format((~grand.SNCZI.isnull()).sum()))


grand[['SNCZI', 'SNCZI_name']] = np.nan
for snczi, name in tqdm(cedex.DAM_NAME.items()):
    
    one = process.extractOne(name, grand.DAM_NAME, score_cutoff=80)

    if one is None:
        continue

    # compute distance
    point1 = cedex.loc[cedex.DAM_NAME == name].geometry.iloc[0]
    point2 = grand.loc[grand.DAM_NAME == one[0]].geometry.iloc[0]
    dist = geodesic((point1.y, point1.x), (point2.y, point2.x)).meters
    if dist < 1e4:
        grand.loc[grand.DAM_NAME == one[0], ['SNCZI', 'SNCZI_name']] = snczi, name
        
print('{0} matches found'.format((~grand.SNCZI.isnull()).sum()))
```

### Reservoir use

In [ ]:
# rename columns
rename_uses_grand = {'USE_IRRI': 'irrigation',
                     'USE_ELEC': 'hydropower',
                     'USE_SUPP': 'supply',
                     'USE_FCON': 'flood',
                     'USE_RECR': 'recreation',
                     'USE_NAVI': 'navigation',
                     'USE_FISH': 'fish',
                     'USE_PCON': 'pollution',
                     'USE_LIVE': 'livestock',
                     'USE_OTHR': 'other'}
grand.rename(columns=rename_uses_grand, inplace=True)

# replace values in 'MAIN_USE' to be consistent with the columns
grand.MAIN_USE.replace({'Irrigation': 'irrigation',
                        'Hydroelectricity': 'hydropower',
                        'Water supply': 'supply',
                        'Other': 'other',
                        'Recreation': 'recreation',
                        'Flood control': 'flood'},
                       inplace=True)

# convert to boolean
grand_uses = list(rename_uses_grand.values())
grand[grand_uses] = grand[grand_uses].replace({'Main': True, 'Major': True, 'Sec': True, None: False})

# identify single purpose reservoirs
grand['SINGLE_USE'] = False
mask_singleuse = grand[grand_uses].sum(axis=1) == 1
grand.loc[mask_singleuse, 'SINGLE_USE'] = True

**Main use**

In [ ]:
mask_singleuse = grand.SINGLE_USE
mask_hdcc = ~grand.index.isnull()

mainuse_singlepurpose = grand.loc[mask_hdcc & mask_singleuse, 'MAIN_USE'].value_counts()
mainuse_multipurpose = grand.loc[mask_hdcc, 'MAIN_USE'].value_counts()
mainuse_summary_grand = pd.concat((mainuse_singlepurpose, mainuse_multipurpose), axis=1)
mainuse_summary_grand.columns = ['singlepurpose', 'multipurpose']
mainuse_summary_grand.sort_values('multipurpose', ascending=False, inplace=True)
del mainuse_singlepurpose, mainuse_multipurpose

**All uses**

In [ ]:
alluses_singlepurpose = grand.loc[mask_hdcc & mask_singleuse, grand_uses].sum()
alluses_multipurpose = grand.loc[mask_hdcc, grand_uses].sum()
alluses_summary_grand = pd.concat((alluses_singlepurpose, alluses_multipurpose), axis=1)
alluses_summary_grand.columns = ['singlepurpose', 'multipurpose']
alluses_summary_grand.sort_values('multipurpose', ascending=False, inplace=True)
alluses_summary_grand.dropna(axis=0, how='all', inplace=True)
alluses_summary_grand = alluses_summary_grand[~(alluses_summary_grand == 0).all(axis=1)]
del alluses_singlepurpose, alluses_multipurpose

## Comparison of data sets

In [ ]:
# limit `cedex` to the reservoirs that fulfil the conditions
cedex = cedex.loc[mask_cedex]

### Reservoir volume

* CEDEX dams: `'V_MNL'`. This value does not represent total capacity, but the capacity at a normal level. Therefore, the actual reservoir capacity should always be larger than this value.
* CEDEX reservoirs: `'V_total'`, `'V_live'`
* ICOLD: `'Res_Vol'`
* EFAS: `'Vtotal'`
* GRanD: `'CAP_MCM'`

In [ ]:
variable = 'CAP_MCM'

# combine all volume data in a single table
volume = pd.concat((cedex[[variable, 'VOL_MNL']],
                    efas[variable],
                    icold[variable],
                    grand[variable], 
                   ), axis=1)
volume.columns = ['CEDEX', 'CEDEX_MNL', 'EFAS', 'ICOLD', 'GRanD']

print('Number of reservoirs:\t{0}\nNumber of data sets:\t{1}'.format(*volume.shape))

# summarize
volume_summary = pd.concat(((~volume.isnull()).sum(), volume.sum()), axis=1)
volume_summary.columns = ['no_reservoirs', 'total_volume']
volume_summary.sort_values('no_reservoirs', ascending=False, inplace=True)

# plot summary
fig, axes = plt.subplots(ncols=2, figsize=(8.5, 4), sharey=True)
for ax, col in zip(axes, volume_summary):
    ax.barh(volume_summary.index, volume_summary[col], color='lightsteelblue')
    ax.spines[['top', 'left', 'right', 'bottom']].set_visible(False)
    ax.tick_params(axis='y', length=0)
    if ax == axes[0]:
        ax.set(xlabel='no. reservoirs')
    elif ax == axes[1]:
        ax.set(xlabel='total volume (hm3)')
        
plt.savefig(PATH_PLOTS / 'reservoir_volume_barplot.jpg', dpi=300, bbox_inches='tight');

The data in the plots above only represents the reservoirs in the datasets (EFAS, GRanD, ICOLD) that I could match with reservoirs in the HDCC database. The datasets on their own may contain more reservoirs; for instance, ICOLD includes 1066 reservoirs in Spain, but "only" 374 could be matched with the 394 reservoirs in HDCC.

EFAS and GRanD are the two data sets with fewer number of reservoirs. However, the total volume that those reservoirs represent is very close to more comprehensive data sets as ICOLD or CEDEX. Interestingly, the total volume in ICOLD slightly exceeds that in CEDEX. This could be caused by the fact that CEDEX volume does not represent total capacity, but the storage at the normal reservoir level.

In [ ]:
# pair plot
pair_plot = sns.pairplot(volume, corner=True, plot_kws={'alpha': .5})

# reference 1:1 line
nrow, ncol = pair_plot.axes.shape
vmin, vmax = volume.min().min(), volume.max().max()
for i in range(nrow):
    for j in range(ncol):
        if i > j:
            pair_plot.axes[i, j].plot([vmin, vmax], [vmin, vmax],
                                      c='k', lw=1, ls=':', zorder=0)
pair_plot.fig.suptitle('Reservoir volume (hm3)')

plt.savefig(PATH_PLOTS / 'reservoir_volume_pairplot.jpg', dpi=300, bbox_inches='tight');

I assume that CEDEX `CEDEX` is the reference data set, as it is the local data set that probably was used by GRanD and ICOLD to develop their global data sets.

Overall, there's a good agreement between all data sets:
 * ICOLD has a few reservoirs whose storage clearly exceeds the values reported by CEDEX.
 * GRanD reservoir storage shows more deviations compared with CEDEX. This deviations are small in most cases, and both overestimate or underestimate storage.
 * As the majority of reservoirs in EFAS come from GRanD, the analysis is similar to that of GRanD.
 * The live volume in CEDEX reservoirs is obviously smaller than the normal storage in most of the cases. However, there are a couple of errors in which the live volume exceeds the normal volume.

### Reservoir surface area

All reservoir area values are in km2:

* CEDEX dams: `'A_MNL'` in ha. This value does not represent the maximum area, but the surface at a normal level.
* CEDEX reservoirs: `'AREA'`
* ICOLD: `'Res_Area'`
* EFAS: none
* GRanD: `'AREA_SKM'`

In [ ]:
variable = 'AREA_SKM'

# combine all volume data in a single table
area = pd.concat((cedex[[variable, 'AREA_MNL']],
                  icold[variable],
                  #efas[variable],
                  grand[variable], 
                 ), axis=1)
area.columns = ['CEDEX', 'CEDEX_MNL', 'ICOLD', 'GRanD']

print('Number of reservoirs:\t{0}\nNumber of data sets:\t{1}'.format(*area.shape))

# summarize
area_summary = pd.concat(((~area.isnull()).sum(), area.sum()), axis=1)
area_summary.columns = ['no_reservoirs', 'total_area']
area_summary.sort_values('no_reservoirs', ascending=False, inplace=True)

# plot summary
fig, axes = plt.subplots(ncols=2, figsize=(8.5, 4), sharey=True)
for ax, col in zip(axes, area_summary):
    ax.barh(area_summary.index, area_summary[col], color='lightsteelblue')
    ax.spines[['top', 'left', 'right', 'bottom']].set_visible(False)
    ax.tick_params(axis='y', length=0)
    if ax == axes[0]:
        ax.set(xlabel='no. reservoirs')
    elif ax == axes[1]:
        ax.set(xlabel='total area (km²)')
        
plt.savefig(PATH_PLOTS / 'reservoir_area_barplot.jpg', dpi=300, bbox_inches='tight');

The values for ICOD and CEDEX are very similar, both in terms of number of reservoirs and the total reservoir area. As seen for volume, ICOLD has a slightly smaller set of reservoirs than CEDEX, but the total aggregation slightly exceeds CEDEX. Again, this could be caused by CEDEX reporting area/volume at normal reservoir level, not at its maximum level.

GRanD reports reservoir area in approximately half of the reservoirs. The total reservoir area seems to be in a similar proportion.

In [ ]:
# pair plot
pair_plot = sns.pairplot(area, corner=True, plot_kws={'alpha': .5})

# reference 1:1 line
nrow, ncol = pair_plot.axes.shape
amin, amax = area.min().min(), area.max().max()
for i in range(nrow):
    for j in range(ncol):
        if i > j:
            pair_plot.axes[i, j].plot([amin, amax], [amin, amax],
                                      c='k', lw=1, ls=':', zorder=0)
pair_plot.fig.suptitle('Reservoir surface (km²)')

plt.savefig(PATH_PLOTS / 'reservoir_area_pairplot.jpg', dpi=300, bbox_inches='tight');

The pair plots above show that GRanD underestimates reservoir area., and there seems to be a clear trend (even though the spread increases with increasing reservoir size).

ICOLD data agrees appropriately with CEDEX.

### Catchment area

All catchment area values are in km2?

* CEDEX dams: `'catch_area'` in ha. This value does not represent the maximum area, but the surface at a normal level.
* ICOLD: `'Catch_Area'`
* EFAS: none
* GRanD: `'CATCH_SKM'`

In [ ]:
variable = 'CATCH_SKM'

# combine all volume data in a single table
catchment = pd.concat((cedex[variable],
                       efas[variable],
                       icold[variable],
                       grand[variable], 
                      ), axis=1)
catchment.columns = ['CEDEX', 'EFAS', 'ICOLD', 'GRanD']

print('Number of reservoirs:\t{0}\nNumber of data sets:\t{1}'.format(*catchment.shape))

# summarize
catchment_summary = pd.concat(((~catchment.isnull()).sum(), catchment.sum()), axis=1)
catchment_summary.columns = ['no_reservoirs', 'total_area']
catchment_summary.sort_values('no_reservoirs', ascending=False, inplace=True)

# plot summary
fig, axes = plt.subplots(ncols=2, figsize=(8.5, 4), sharey=True)
for ax, col in zip(axes, catchment_summary):
    ax.barh(catchment_summary.index, catchment_summary[col], color='lightsteelblue')
    ax.spines[['top', 'left', 'right', 'bottom']].set_visible(False)
    ax.tick_params(axis='y', length=0)
    if ax == axes[0]:
        ax.set(xlabel='no. reservoirs')
    elif ax == axes[1]:
        ax.set(xlabel='total catchment area (km²)')
        
plt.savefig(PATH_PLOTS / 'reservoir_catchment_barplot.jpg', dpi=300, bbox_inches='tight');

The catchment area value is missing from several reservoirs in CEDEX, which causes that ICOLD is the most comprehensive data set. Even though the number of reservoirs in GRanD that report catchment area is notably smaller, the total catchment area is close to other data sets, meaning that GRanD includes the reservoirs with larger contributing area.

In [ ]:
# pair plot
pair_plot = sns.pairplot(catchment, corner=True, plot_kws={'alpha': .5})

# reference 1:1 line
nrow, ncol = pair_plot.axes.shape
amin, amax = catchment.min().min(), catchment.max().max()
for i in range(nrow):
    for j in range(ncol):
        if i > j:
            pair_plot.axes[i, j].plot([amin, amax], [amin, amax],
                                      c='k', lw=1, ls=':', zorder=0)
pair_plot.fig.suptitle('Catchment area (km²)')
            
plt.savefig(PATH_PLOTS / 'reservoir_catchment_pairplot.jpg', dpi=300, bbox_inches='tight');

There is a considerable scatter in the areas among the smallest catchments. Specifically, GRanD has some rerservoirs which overestimate catchment area compared both with CEDEX and ICOLD. ICOLD, instead, includes catchment areas that differ in both ways (over or underestimate) the area of small catchments compared with CEDEX.

**Test GIS and `cutmaps` catchments**

In [ ]:
# calculate the upstream area for each reservoir catchment
filename = 'upArea_01min.nc'
path_cutmaps = PATH_RESOPSES / 'ancillary' / 'cutmaps'
cutmaps = pd.Series(index=points.id, dtype=float, name='CATCH_SKM')
cutmaps.index.name = 'SNCZI'
for ID in tqdm(cutmaps.index):
    # load map cut
    data = xr.open_dataset(path_cutmaps / ID / filename)['Band1']
    # calculate catchment area in km²
    cutmaps.loc[ID] = data.max().values / 1e6

# assign values to the "catchment" DataFrame
cutmaps.index = cutmaps.index.astype(int)
catchment.loc[cutmaps.index, 'cutmaps'] = cutmaps.values

In [ ]:
# import watershed polygons derived in GIS
gis = gpd.read_file(PATH_RESOPSES / 'GIS' / 'Layers' / 'watershed_305.shp')
gis.Name = gis.Name.astype('Int64')
gis.rename(columns={'Name': 'SNCZI', 'Shape_Leng': 'LENG_KM', 'Shape_Area': 'CATCH_SKM'}, inplace=True)
gis.set_index('SNCZI', inplace=True)
gis.drop(['OID_', 'HydroID'], axis=1, inplace=True)

# assign values to the "catchment" DataFrame
catchment.loc[gis.index, 'GIS'] = gis.CATCH_SKM.values

In [ ]:
sns.pairplot(catchment, corner=True, plot_kws={'alpha': .5});

In [ ]:
catchment.loc[[616, 1038, 1290, 1342, 1643, 1840, 1953, 2050]]

In [ ]:
idx = reservoirs.loc[reservoirs.TS == 1].index.difference(watershed.index).tolist()

In [ ]:
reservoirs.loc[idx]

In [ ]:
watershed.boundary.plot()

In [ ]:
# add to the comparative table the area of the polygons derived in GIS
catchment.loc[watershed.index, 'GIS'] = watershed['CATCH_SKM']

In [ ]:
# # create a field reporting the most reliable value of the catchment area
# catchment['CATCH_SKM'] = catchment[['CEDEX', 'ICOLD']].max(axis=1)
# mask = catchment.CATCH_SKM.isnull()
# catchment.loc[mask, 'CATCH_SKM'] = catchment.loc[mask, 'GRanD']

In [ ]:
# create a field reporting the most reliable value of the catchment area
catchment['CATCH_SKM'] = catchment.GRanD
mask = catchment.CATCH_SKM.isnull()
catchment.loc[mask, 'CATCH_SKM'] = catchment.loc[mask, ['CEDEX', 'ICOLD']].min(axis=1)

In [ ]:
error = (catchment.GIS - catchment.CATCH_SKM) / catchment.CATCH_SKM * 100
catchment.loc[error.abs() > 50].sort_index().round(0)

It seems like GRanD is a more reliable source in terms of catchment area, so, when available, I compare against this data set. If not avaible, I compare agains the minimum between CEDEX and ICOLD.

* __Correct in GIS__: 52, 173, 317, 400, 442, 620, 916, 993, 1019, 1062, 1075, 1125, 1285, 1565, 1657, 2083, 2101, 2175, 2261, 2469?, 2561
* __Wrong in GIS__: 616, 1038, 1290, 1342, 1643, 1840, 1953, 2050

Incongruences between CEDEX and ICOLD.

In [ ]:
print(f'{catchment.CEDEX.isnull().sum()} reservoir miss the catchment area in CEDEX')
print(f'{catchment.ICOLD.isnull().sum()} reservoir miss the catchment area in ICOLD')
print(f'{catchment.GRanD.isnull().sum()} reservoir miss the catchment area in GRanD')
print(f'{catchment.GIS.isnull().sum()} reservoir miss the catchment area in GIS')

### Reservoir use

**Main use**

In [ ]:
main_use = {'CEDEX': mainuse_summary_cedex,
            'ICOLD': mainuse_summary_icold,
            'GRanD': mainuse_summary_grand}

fig, axes = plt.subplots(ncols=len(main_use), figsize=(len(main_use) * 4, 4), sharex=True, tight_layout=True)
for ax, (label, df) in zip(axes, main_use.items()):
    ax.barh(df.index, df.multipurpose, color='steelblue', label='multi-purpose')
    ax.barh(df.index, df.singlepurpose, color='lightsteelblue', label='single purpose')
    ax.set(title=label,
           xlabel='no. reservoirs')
    ax.tick_params(axis='y', length=0)
    ax.spines[['top', 'left', 'right', 'bottom']].set_visible(False);
fig.suptitle('Main reservoir use')
fig.legend(*ax.get_legend_handles_labels(), loc=5, frameon=False, bbox_to_anchor=[1.02, .4, .1, .2]);

plt.savefig(PATH_PLOTS / 'reservoir_main_use.jpg', dpi=300, bbox_inches='tight');

The definition of the main use of a reservoir is ambiguous. Is it the use that requires larger water volume? Is it a design decision? 

The ambiguity is clear in the plot above. Even though the two most common main uses (supply and hydropower) coincide in CEDEX and ICOLD, the figures differ notably. These two data sets assign different importance to flood and irrigation. On the contrary, GRanD identifies irrigation as the main reservoir use in Spain, followed by hydropower and supply; flood control has no importance in this data set.

**All uses**

In [ ]:
all_uses = {'CEDEX': alluses_summary_cedex,
            'ICOLD': alluses_summary_icold,
            'GRanD': alluses_summary_grand}

fig, axes = plt.subplots(ncols=len(all_uses), figsize=(len(all_uses) * 4, 4), sharex=True, tight_layout=True)
for ax, (label, df) in zip(axes, all_uses.items()):
    ax.barh(df.index, df.multipurpose, color='steelblue', label='multi-purpose')
    ax.barh(df.index, df.singlepurpose, color='lightsteelblue', label='single purpose')
    ax.set(title=label,
           xlabel='no. reservoirs')
    ax.tick_params(axis='y', length=0)
    ax.spines[['top', 'left', 'right', 'bottom']].set_visible(False);
fig.suptitle('All reservoir uses')
fig.legend(*ax.get_legend_handles_labels(), loc=5, frameon=False, bbox_to_anchor=[1.02, .4, .1, .2]);

plt.savefig(PATH_PLOTS / 'reservoir_all_uses.jpg', dpi=300, bbox_inches='tight');

The difference between the three data sets remains when looking at all the reservoir uses. ICOLD and GRanD identify hydropower and irrigation as the two most common uses, even though the order differs between them two. CEDEX, instead, considers water supply as the main use (3rd in ICOLD and GRanD), and limits hydropower and irrigation to the 3rd and 4th rank. It's remarkable that flood control is the second most common use in CEDEX, but it has a residual value in both ICOLD and GRanD.

## Export

In [ ]:
map_snczi_grand = grand.GRAND_ID.to_dict()

In [ ]:
len(map_snczi_grand), cedex.shape[0], grand.shape[0]

### CEDEX

In [ ]:
# set GRAND_ID as index
cedex.reset_index(inplace=True)
cedex['GRAND_ID'] = cedex.SNCZI.map(map_snczi_grand)
cedex = cedex[cedex.GRAND_ID.notnull()]
cedex.GRAND_ID = cedex.GRAND_ID.astype(int)
cedex.set_index('GRAND_ID', inplace=True)
print(f'{len(cedex)} CEDEX reservoirs have a GRAND_ID')

# convert boolean fields to 0-1
bool_cols = list(rename_uses_cedex.values()) + ['single_use']
cedex[bool_cols] = cedex[bool_cols].astype(bool).astype(int)

# reorder columns
cols_cedex = cedex.columns.tolist()
cols_cedex.remove('RES_NAME')
cols_cedex = ['RES_NAME'] + cols_cedex
cedex = cedex[cols_cedex]

# export
cedex.drop(columns=['geometry']).to_csv(cfg.PATH_ATTRS / 'cedex.csv')

### EFAS

In [ ]:
# set GRAND_ID as index
efas.reset_index(inplace=True)
efas['GRAND_ID'] = efas.SNCZI.map(map_snczi_grand)
efas = efas[efas.GRAND_ID.notnull()]
efas.GRAND_ID = efas.GRAND_ID.astype(int)
efas.set_index('GRAND_ID', inplace=True)
print(f'{len(efas)} EFAS reservoirs have a GRAND_ID')

# rename columns
efas.rename(columns={'ACTIVE_FRO': 'YEAR'}, inplace=True)

# attributes to be included from EFAS
cols_efas = ['SNCZI', 'DAM_NAME', 'YEAR',
             # coordinates
             'LON', 'LAT', 'LisfloodX', 'LisfloodY',
             # LISFLOOD limits
             'Vc', 'Vf', 'Qmin', 'Qnd', 'Vn', 'Qn', 'CAP_MCM',
             # catchment area
            'CATCH_SKM']

# export as CSV
efas[cols_efas].to_csv(cfg.PATH_ATTRS / 'efas.csv')

### ICOLD

In [ ]:
# set GRAND_ID as index
icold.reset_index(inplace=True)
icold['GRAND_ID'] = icold.SNCZI.map(map_snczi_grand)
icold = icold[icold.GRAND_ID.notnull()]
icold.GRAND_ID = icold.GRAND_ID.astype(int)
icold.set_index('GRAND_ID', inplace=True)
print(f'{len(icold)} ICOLD reservoirs have a GRAND_ID')

# convert 'SNCZI' to integer
# icold.SNCZI = icold.SNCZI.astype('Int64')
# icold.set_index('SNCZI', inplace=True)

# convert boolean fields to 0-1
bool_cols = list(rename_uses_icold.values()) + ['single_use']
icold[bool_cols] = icold[bool_cols].astype(bool).astype(int)

# attributes to be included from ICOLD
cols_icold = ['SNCZI', 'RES_NAME', 'DAM_NAME', 'ELEV_MASL', 'CATCH_SKM', 'DAM_LEN_M', 
              'Elec_Capac', 'Ene_Annual',
              'Foundation', 'DAM_HGT_M',
              'Irr_Area', 
              'LON', 'LAT',
              'AREA_SKM', 'RES_LEN_KM', 'CAP_MCM', 'Resettled',
              'Spill_Cap', 'Spill_Type', 'Vol_flood', 'Year',
              'main_use', 'single_use', 'fish', 'flood', 'hydropower', 'irrigation', 'navigation', 'other', 'recreation', 'supply', 'tailings']

# export attribute table
icold[cols_icold].to_csv(cfg.PATH_ATTRS / 'icold.csv')

### GRanD

In [ ]:
# set GRAND_ID as index
grand.reset_index(inplace=True)
grand.set_index('GRAND_ID', inplace=True)
print(f'{len(grand)} reservoirs in GRanD')

# convert boolean fields to 0-1
bool_cols = list(rename_uses_grand.values()) + ['SINGLE_USE']
grand[bool_cols] = grand[bool_cols].astype(bool).astype(int)

# attributes to be included from EFAS
cols_grand = ['SNCZI', 'RES_NAME', 'DAM_NAME', 'YEAR', 'REM_YEAR', 'LON', 'LAT',
              'DAM_HGT_M', 'DAM_LEN_M', 'AREA_SKM', 'CAP_MCM', 'DEPTH_M', 'DIS_AVG_LS', 'DOR_PC', 'ELEV_MASL', 'CATCH_SKM',
              'irrigation', 'hydropower', 'supply', 'flood', 'recreation', 'navigation', 'fish', 'pollution', 'livestock', 'other_use', 'MAIN_USE', 'SINGLE_USE','LAKE_CTRL']

# export as CSV
grand[cols_grand].to_csv(cfg.PATH_ATTRS / 'grand.csv')

## Conclusion

ICOLD seems to be the best data set to extract the reservoir static attributes. In the specific case of Spain, the Spanish National Inventory of Reservoirs and Dams (CEDEX) is more comprehensive, both in terms of number of reservoirs and the number of attributes. However, since this is only an exercise that must be extrapolated continentally/globally, we need to use a global data set like ICOLD or GRanD. The comparison of these two global data sets against CEDEX shows that the figures in ICOLD are closer to the reference. On top of that, I could match 374 reservoirs between the observed time series and ICOLD, and only 225 with GRanD, so using ICOLD increases notably the sample of reservoirs.

The drawback of using ICOLD is that the data can not be open.

In [ ]:
# export shapefile of reservoirs
hdcc['lon_orig'] = hdcc.geometry.x
hdcc['lat_orig'] = hdcc.geometry.y
hdcc[['lon_lisf', 'lat_lisf']] = np.nan

# hdcc.to_file(PATH_RESOPSES / 'GIS' / 'reservoirs_ResOpsES.shp')

### Reservoirs with time series

**Anuario de Aforos**

Import shapefile of reservoirs in the Hydrological Data Colection Center (HDCC) database.

In [ ]:
path_cedex = PATH_DATASETS / 'CEDEX' / 'processed' / 'reservoirs'
reservoirs_AA = gpd.read_file(path_cedex / 'attributes' / 'GIS' / 'reservoirs.shp')
reservoirs_AA.ref_ceh = reservoirs_AA.ref_ceh.astype('Int64')
reservoirs_AA['source'] = 'CEDEX'
reservoirs_AA.rename(columns={'nombre': 'name', 'ref_ceh': 'ID'}, inplace=True)
reservoirs_AA['SNCZI'] = reservoirs_AA['SNCZI'].replace(0, np.nan).astype('Int64')
reservoirs_AA.set_index('SNCZI', drop=True, inplace=True)
reservoirs_AA.sort_index(axis=0, inplace=True)

print('Anuario de Aforos contains {0} reservoirs'.format(reservoirs_AA.shape[0]))
print('{0} reservoirs are not connected to the SNCZI database'.format((reservoirs_AA.index.isnull()).sum()))
# reservoirs_AA.head()

**Agència Catalana de l'Aigua**

I have downloaded reservoir records from the ACA website, but these reservoirs are not yet in HDCC. I will upload here the shapefile of the ACA reservoirs and concatenate it to HDCC.

In [ ]:
# load shapefile and treat the data
reservoirs_aca = gpd.read_file(PATH_DATASETS / 'ACA' / 'processed' / 'reservoirs' / 'attributes' / 'GIS' / 'reservoirs_ACA.shp')
reservoirs_aca['source'] = 'ACA'
reservoirs_aca.set_index('SNCZI', inplace=True, drop=True)
reservoirs_aca.rename(columns={'ID_ACA': 'ID'}, inplace=True)

print('ACA contains {0} reservoirs'.format(reservoirs_aca.shape[0]))
print('{0} reservoirs are not connected to the SNCZI database'.format((reservoirs_aca.index.isnull()).sum()))
# reservoirs_aca.head()

**Hidrosur**

I have requested reservoir data from Hidrosur. This data is not yet in HDCC, so I will load it here and combine it with the reservoirs in HDCC.

In [ ]:
# load shapefile and treat the data
reservoirs_sur = gpd.read_file(PATH_DATASETS / 'Hidrosur' / 'processed' / 'reservoirs' / 'attributes' / 'GIS' / 'reservoirs_hidrosur.shp')
reservoirs_sur['source'] = 'Hidrosur'
reservoirs_sur['SNCZI'] = reservoirs_sur['SNCZI'].astype('Int64')
reservoirs_sur.set_index('SNCZI', drop=True, inplace=True)

print('Hidrosur contains {0} reservoirs'.format(reservoirs_sur.shape[0]))
print('{0} reservoirs are not connected to the SNCZI database'.format((reservoirs_sur.index.isnull()).sum()))
# reservoirs_sur.head()

**Merge all**

<font color='indianred'>I remove the reservoirs without SNCZI code.</font>

In [ ]:
cols = ['ID', 'name', 'source', 'geometry']
hdcc = pd.concat((reservoirs_AA[cols], reservoirs_aca[cols], reservoirs_sur[cols]), axis=0)
hdcc.index = hdcc.index.astype('Int64')
# remove one of the instances of the Siurana reservoir (included both in Anuario and ACA)
# I remove the instance in the ACA (`ID == 'E18')
hdcc = hdcc.loc[hdcc.ID != 'E18']
hdcc.name = hdcc.name.str.replace(r'(.+), (.+)', r'\2 \1', regex=True)
# hdcc.name = hdcc.name.apply(unidecode.unidecode)
hdcc.name = [remove_accents(name) for name in hdcc.name]

print('Anuario + ACA + Hidrosur:\t{0} reservoirs'.format(hdcc.shape[0]))
print('{0} reservoirs are not connected to the SNCZI database'.format((hdcc.index.isnull()).sum()))

hdcc = hdcc.loc[~(hdcc.index.isnull())]

fig, ax = plt.subplots(subplot_kw=dict(projection=ccrs.PlateCarree()))
ax.add_feature(cfeature.NaturalEarthFeature('physical', 'land', '10m', edgecolor='face', facecolor='lightgray'),
               alpha=.5,
               zorder=0)
hdcc.plot(markersize=5, ax=ax)
ax.set_title(f'Hydro DB\n{hdcc.shape[0]} reservoirs')
ax.set_extent([-10, 4.5, 35.5, 44])
ax.axis('off');

### Select reservoirs

In [ ]:
masks = {'CEDEX': mask_cedex,
         'EFAS': mask_efas,
         'ICOLD': mask_icold,
         'GRanD': mask_grand}

attributes = {'CEDEX': cedex,
              'EFAS': efas,
              'ICOLD': icold,
              'GRanD': grand}

summary = pd.DataFrame(index=attributes.keys(), columns=['initially', 'selected', 'timeseries'])
for label, df in attributes.items():
    summary.loc[label, 'initially'] = masks[label].shape[0]
    summary.loc[label, 'selected'] = df.shape[0]
    summary.loc[label, 'timeseries'] = len(df.index.intersection(hdcc.index))
    
total_IDs = []
for label, serie in masks.items():
    total_IDs += serie.index.tolist()
total_IDs = set(total_IDs)
summary.loc['total', 'initially'] = len(total_IDs)
    
selected_IDs = []
for label, df in attributes.items():
    selected_IDs += df.index.tolist()
selected_IDs = set(selected_IDs)
summary.loc['total', 'selected'] = len(selected_IDs)
summary.loc['total', 'timeseries'] = len(selected_IDs.intersection(hdcc.index))

summary

In [ ]:
reservoirs = pd.DataFrame(columns=['RES_NAME', 'DAM_NAME', 'LON', 'LAT', 'CAP_MCM', 'AREA_SKM', 'CATCH_SKM'])
reservoirs.index.name = 'SNCZI'

# add reservoirs from CEDEX
reservoirs = pd.concat((reservoirs, cedex[reservoirs.columns]))

# add reservoirs from ICOLD
sel = icold.index.difference(reservoirs.index)
reservoirs = pd.concat((reservoirs, icold.loc[sel, reservoirs.columns]))

# add reservoirs from GRanD
sel = grand.index.difference(reservoirs.index)
reservoirs = pd.concat((reservoirs, grand.loc[sel, reservoirs.columns]))

# add reservoirs from EFAS
sel = efas.index.difference(reservoirs.index)
reservoirs = pd.concat((reservoirs, efas.loc[sel, ['DAM_NAME', 'LON', 'LAT', 'CAP_MCM', 'CATCH_SKM']]))

# time series available?
# reservoirs['TS'] = 0
idx_ts = reservoirs.index.intersection(hdcc.index)
# reservoirs.loc[idx_ts, 'TS'] = 1
reservoirs.loc[idx_ts, ['SOURCE_TS', 'ID_TS']] = hdcc.loc[idx_ts, ['source', 'ID']].values

# convert into GeoDataFrame
geometry = [Point(xy) for xy in zip(reservoirs.LON, reservoirs.LAT)]
reservoirs = gpd.GeoDataFrame(reservoirs, geometry=geometry, crs='epsg:4326')

# export point shapefile
reservoirs.loc[reservoirs.ID_TS.notnull()].to_file(PATH_RESOPSES / 'GIS' / 'reservoirs_ResOpsES.shp')

reservoirs.head()

***

In [ ]:
# create TXT file to apply CUTMAPS
path_cutmaps = PATH_RESOPSES / 'ancillary' / 'cutmaps' / 'input'
points = gpd.read_file(path_cutmaps / 'reservoirs.shp')
points = points[['LON', 'LAT', 'SNCZI']]
points[['LON', 'LAT']] = points[['LON', 'LAT']].astype(float).round(4)
points['SNCZI'] = [f'{ID:04}' for ID in points.SNCZI]

points.to_csv(path_cutmaps / 'reservoirs.txt',
              sep='\t',
              header=False,
              index=False,
              float_format='%.4f')

***

In [ ]:
# create CSV file to apply NCEXTRACT
path_ncextract = PATH_RESOPSES / 'ancillary' / 'ncextract'
points = gpd.read_file(path_ncextract / 'reservoirs.shp')
points = points[['LON', 'LAT', 'SNCZI']]
points[['LON', 'LAT']] = points[['LON', 'LAT']].astype(float).round(4)
points['SNCZI'] = [f'{ID:04}' for ID in points.SNCZI]
points.rename(columns={'SNCZI': 'id', 'LAT': 'lat', 'LON': 'lon'}, inplace=True)

points[['id', 'lat', 'lon']].to_csv(path_ncextract / 'reservoirs.csv',
                                    header=True,
                                    index=False,
                                    float_format='%.4f')

***